# Hello F#

This is a .net interactive notebook with some sample code of the basic of F# that you play around with.

Things we will cover in this notebook is:

* Simple types
* What is `let`
* More complex types
* Pattern matching

## Simple types

Types is one of the most important strength in F#. I've devided types into two sections, one around simple types and then one more advanced types later on.

Types are defined using the `type` keyword. That is used for all the different types you can define, records, discriminated unions, classes, abstract classes or interfaces (yes you can do classes and interfaces in F#)

Let's see some examples

In [ ]:
// Type alias 
type Age = int // This is just an alias over a native built-in type

// Record
type Person = {
    Name: string
    Age: int
}

// Discriminated union
type Employee =
    | Manager of Person
    | Employee of Person

## All you need is `let`

`let` is the keyword used to define and assign a variable in F#. You use it to define functions and values. One thing that is important in F# is that everything is an expression. So everything has an output.

In [ ]:
// Integer
let x = 10
// String
let y = "Hello"
// Record 
let tomas = { Name = "Tomas"; Age = 30 }
let tomas2 = {
    Name = "Tomas" // No need for ;
    Age = 30
}
// DUs
let employee = Employee tomas
let manager = Employee tomas2

// Functions
let add x y = x + y
add 5 6
// add 5.5 5.6 // Fails since add has been inferred to be of type int

11

In [ ]:
// More functions
let changeName name person = { person with Name = name }
changeName "Ole" tomas


Name,Age
Ole,30


In [ ]:
tomas // is still unchanged

Name,Age
Tomas,30


In [ ]:
let changeEmployeeName name employee = 
    match employee with
    | Manager person -> Manager (changeName name person)
    | Employee person -> Employee (changeName name person)

changeEmployeeName "ole" manager

Item
"{ { Name = ""ole"" Age = 30 }: Name: ole, Age: 30 }"


## How do you work with "null"?

> Null References: The Billion Dollar Mistake - Tony Hoare (creator of null)

In "pure" F# null isn't allowed, so how do you work with null?

The answer is `Option`. If you want to make something optional, you must define it as an `Option` which forces you to do the right thing to get the code to compile.

In [ ]:
type Name = { FirstName: string; LastName: string; MiddleName: string option }
let name1 = { FirstName = "Tomas"; LastName = "Jansson"; MiddleName = None }
let name2 = { FirstName = "Tomas"; LastName = "Jansson"; MiddleName = Some "Doe" }

let printName name =
    match name.MiddleName with
    | None -> sprintf "%s %s" name.FirstName name.LastName
    | Some middleName -> sprintf "%s (%s) %s" name.FirstName middleName name.LastName

printName name1, printName name2


Item1,Item2
Tomas Jansson,Tomas (Doe) Jansson


## Currying and partial applications

Currying and partial applications are central concept in programming in general, and you can also use it in most languages today that treats functions as first class citizens (C# does as well).

Currying is to allow a function that takes multiple arguments to be written as a sequence of functions where each function takes one argument.

    let add1 (x,y) = x + y // function that takes to arguments
    let add2 x y = x + y // curried version

Partial applications is what you got when you call a curried function with only some of its values

    let add100 = add2 100 // This will call add2 with argumenet 100 returning a new function where x is set to be 100

In [ ]:
let add1 (x,y) = x + y
let add2 x y = x + y
let add100 = add2 100

(add1(5,4), add 5 4, add100 5)

Item1,Item2,Item3
9,9,105


In [ ]:
let doMath operator x y = operator x y
let doAddition = doMath (fun x y -> x + y)
let doSubtraction = doMath (fun x y -> x - y)
let doMultiplication = doMath (fun x y -> x * y)
let doDivision = doMath (fun x y -> x / y)

doAddition 10 10, doSubtraction 10 4, doMultiplication 10 10, doDivision 100 10

Item1,Item2,Item3,Item4
20,6,100,10


## Piping (|>)

Piping is very commong in F#, and it is a lot like bash piping. The syntax in short is

    value |> function

this is exactly the same as calling

    function value

In [ ]:
let parse x = int x
let validate x = 
    if x > 2 then x else raise (exn "Invalid number")
let double x = x * 2

let application x =
    x |> parse |> validate |> double

"5" |> application

10

## More types

Discriminated unions is one of the most powerful features in F# (in my opinion). So let's explore them a little bit more.

Basic structure of discriminated unions:

```
type <Name of DU> =
    | <case> (of data)

type Option<'T> =
    | None
    | Some of 'T
```

In [ ]:
type MyOption<'T> =
    | MySome of 'T
    | MyNone

let x = MySome 5
let y = MyNone

let toString x =
    match x with
    | MySome x -> sprintf "%A" x
    | MyNone -> sprintf "None"

(toString x, toString y)

Item1,Item2
5,None


In [ ]:
// More advanced DU

// Use case, create a loyalty engine! :)
// We should be able to define rules based on order line which will say if they should get bonus point or not

// Filter will define if the order line should get bonus point
type Filter<'T> = 
    | And of ('T -> bool)
    | Or of Filter<'T> list
    | Not of Filter<'T>
 
// Sink defins how bonus points are calculated
type Sink<'T> = 'T -> 'T

// Workflow defines how a flow with multiple filters and sinks should be created
type Workflow<'T> =
    | Sink of Sink<'T>
    | Filter of Filter<'T> * Workflow<'T>

// In the real world this will be part of order, but simplified for the sake of this example
type OrderLineResult = {
    Brand: string
    ArticleId: string
    Amount: int
    Points: int
}

// Sink function that gives double points
let doublePointSink (orderLine) = 
    {orderLine with Points = orderLine.Amount * 2 } 

// Sink function that gives one point per crown
let baseSink (orderLine) =
    {orderLine with Points = orderLine.Amount } 


// Some simple filters
let atLeast5000Filter = And (fun orderLine -> orderLine.Amount >= 5000)
let appleBrandFilter = And (fun orderLine -> orderLine.Brand = "Apple")
let samsungBrandFilter = And (fun orderLine -> orderLine.Brand = "Samsung")

// Combined filter
let appleOrSamsungFilter = Or [appleBrandFilter; samsungBrandFilter]

let workflow = Filter (atLeast5000Filter, (Filter (appleOrSamsungFilter, (Sink doublePointSink))))

let rec execFilter filter orderLine =
    match filter with
    | And func -> func orderLine
    | Or funcs -> funcs |> List.exists (fun func -> execFilter func orderLine)
    | Not func -> not (execFilter func orderLine)

let rec execute workflow orderLine =
    match workflow with
    | Sink sink -> sink orderLine
    | Filter (filter, nextStep) ->
        if execFilter filter orderLine 
        then execute nextStep orderLine
        else orderLine

let appleOrderBelow5000 = { Brand = "Apple"; ArticleId = "A"; Amount = 1000; Points = 0 }
let appleOrderAbove5000 = { Brand = "Apple"; ArticleId = "A"; Amount = 5001; Points = 0 }
let samsungOrderBelow5000 = { Brand = "Samsung"; ArticleId = "S"; Amount = 1000; Points = 0 }
let samsungOrderAbove5000 = { Brand = "Samsung"; ArticleId = "S"; Amount = 5001; Points = 0 }

[
    appleOrderBelow5000
    appleOrderAbove5000
    samsungOrderBelow5000
    samsungOrderAbove5000
] |> List.map (execute workflow)


index,Brand,ArticleId,Amount,Points
0,Apple,A,1000,0
1,Apple,A,5001,10002
2,Samsung,S,1000,0
3,Samsung,S,5001,10002


In [ ]:
let executeMany workflows orderLine =
    workflows
    // Do the calculation for all workflows
    |> List.map (fun workflow -> execute workflow orderLine)
    // Pick the result with the most points
    |> List.maxBy (fun orderLine -> orderLine.Points)

let allWorkflows = [workflow; Sink baseSink]
[
    appleOrderBelow5000
    appleOrderAbove5000
    samsungOrderBelow5000
    samsungOrderAbove5000
] |> List.map (executeMany allWorkflows)

index,Brand,ArticleId,Amount,Points
0,Apple,A,1000,1000
1,Apple,A,5001,10002
2,Samsung,S,1000,1000
3,Samsung,S,5001,10002
